In [1]:
import vaidata
from vaiutils import path_consts
import json
from glob import glob
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
for k, v in path_consts('COCO'):
    exec(k + '=v')

In [3]:
with open(DIR_DATA + '/captions_val2017.json') as f:
    captions = json.load(f)

In [4]:
filenames = sorted(glob(DIR_DATA + '/val2017/*.jpg'))

In [5]:
file_ids = []
for f in filenames:
    red_f = f[f.find('/0') + 1:f.find('.jpg')]
    file_ids.append(int(red_f))

In [6]:
caption_dict = {}
for caption in captions['annotations']:
    if caption['image_id'] in file_ids:
        if caption['image_id'] not in caption_dict.keys():
            caption_dict[caption['image_id']] = []
        caption_dict[caption['image_id']].append(caption['caption'])

In [13]:
simple_caption_list = []
caption_list = []
all_captions = ''
all_simple_captions = ''
for f in file_ids:
    caption_list.append([])
    simple_caption_list.append([])
    for caption in caption_dict[f]:
        simple_caption_list[-1].append(caption)
        f_stop_idx = caption.find('.')
        if f_stop_idx != -1:
            caption_list[-1].append('` ' + caption[:f_stop_idx] + ' ' + caption[f_stop_idx: ])
        else:
            caption_list[-1].append('` ' + caption + ' .')
        all_simple_captions += simple_caption_list[-1][-1] + ' '
        all_captions += caption_list[-1][-1] + ' '

In [14]:
simple_caption_list

[['A woman stands in the dining area at the table.',
  'A room with chairs, a table, and a woman in it.',
  'A woman standing in a kitchen by a window',
  'A person standing at a table in a room.',
  'A living area with a television and a table'],
 ['A big burly grizzly bear is show with grass in the background.',
  'The large brown bear has a black nose.',
  'Closeup of a brown bear sitting in a grassy area.',
  'A large bear that is sitting on grass. ',
  "A close up picture of a brown bear's face."],
 ['Bedroom scene with a bookcase, blue comforter and window.',
  'A bedroom with a bookshelf full of books.',
  'This room has a bed with blue sheets and a large bookcase',
  'A bed and a mirror in a small room.',
  'a bed room with a neatly made bed a window and a book shelf'],
 ["A stop sign is mounted upside-down on it's post. ",
  'A stop sign that is hanging upside down.',
  'An upside down stop sign by the road.',
  'a stop sign put upside down on a metal pole ',
  'A stop sign in

In [16]:
def pickle_dump(filename, obj):
    vaidata.pickle_dump(DIR_DATA + '/' + filename + '.p', obj)

In [17]:
tokenizer = Tokenizer(2000, filters='!"#$%&()*+,-/:;<=>?@[\\]^_{|}~\t\n')
tokenizer.fit_on_texts([all_simple_captions])

caption_dict = {}
for captions, file_id in zip(simple_caption_list, file_ids):
    caption_tokens = tokenizer.texts_to_sequences(captions)
    caption_dict[file_id] = [np.array(caption_token) for caption_token in caption_tokens]

In [19]:
pickle_dump('simple_captions', caption_dict)
pickle_dump('simple_tokenizer', tokenizer)

In [20]:
tokenizer = Tokenizer(2000, filters='!"#$%&()*+,-/:;<=>?@[\\]^_{|}~\t\n')
tokenizer.fit_on_texts([all_captions])

caption_dict = {}
for captions, file_id in zip(caption_list, file_ids):
    caption_tokens = tokenizer.texts_to_sequences(captions)
    caption_dict[file_id] = [np.array(caption_token) for caption_token in caption_tokens]

In [21]:
pickle_dump('captions', caption_dict)
pickle_dump('tokenizer', tokenizer)